**Use GPU: Runtime -> Change runtime type -> GPU (Hardware Accelerator)**

Setup

In [1]:
!cat ~/.keras/keras.json 

{
    "epsilon": 1e-07, 
    "floatx": "float32", 
    "image_data_format": "channels_last", 
    "backend": "tensorflow"
}

Mini-VGGNet

In [2]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

In [3]:
class MiniVGGNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1

		# first CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(32, (3, 3), padding="same", input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(32, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# second CONV => RELU => CONV => RELU => POOL layer set
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

Train Ensemble (MiniVGGNet) w/ CIFAR10 + Data Aug

In [4]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np
import os

In [7]:
def train_ensemble(output_filename, models_folder, num_models=5):
    # load the training and testing data, then scale it into the
    # range [0, 1]
    ((trainX, trainY), (testX, testY)) = cifar10.load_data()
    trainX = trainX.astype("float") / 255.0
    testX = testX.astype("float") / 255.0

    # convert the labels from integers to vectors
    lb = LabelBinarizer()
    trainY = lb.fit_transform(trainY)
    testY = lb.transform(testY)

    # initialize the label names for the CIFAR-10 dataset
    labelNames = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

    # construct the image generator for data augmentation
    aug = ImageDataGenerator(rotation_range=10, width_shift_range=0.1,
        height_shift_range=0.1, horizontal_flip=True,
        fill_mode="nearest")

    # loop over the number of models to train
    for i in np.arange(0, num_models):
        # initialize the optimizer and model
        print("[INFO] training model {}/{}".format(i + 1, num_models))
        opt = SGD(lr=0.01, decay=0.01 / 40, momentum=0.9, nesterov=True)
        model = MiniVGGNet.build(width=32, height=32, depth=3, classes=10)
        model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

        # train the network
        H = model.fit_generator(aug.flow(trainX, trainY, batch_size=64),
            validation_data=(testX, testY), epochs=40,
            steps_per_epoch=len(trainX) // 64, verbose=1)

        # save the model to disk
        p = [models_folder, "model_{}.model".format(i)]
        model.save(os.path.sep.join(p))

        # evaluate the network
        predictions = model.predict(testX, batch_size=64)
        report = classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames)

        # save the classification report to file
        p = [output_filename, "model_{}.txt".format(i)]
        f = open(os.path.sep.join(p), "w")
        f.write(report)
        f.close()

        # plot the training loss and accuracy
        p = [output_filename, "model_{}.png".format(i)]
        plt.figure()
        plt.plot(np.arange(0, 40), H.history["loss"], label="train_loss")
        plt.plot(np.arange(0, 40), H.history["val_loss"], label="val_loss")
        plt.plot(np.arange(0, 40), H.history["accuracy"], label="train_accuracy")
        plt.plot(np.arange(0, 40), H.history["val_accuracy"], label="val_accuracy")
        plt.title("Training Loss and Accuracy for model {}".format(i))
        plt.xlabel("Epoch #")
        plt.ylabel("Loss/Accuracy")
        plt.legend()
        plt.savefig(os.path.sep.join(p))
        plt.close()

In [8]:
train_ensemble(output_filename="drive/MyDrive/pyimagesearch/output/25-ensembles/output", models_folder="drive/MyDrive/pyimagesearch/output/25-ensembles/models")

[INFO] training model 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
781/781 [==============================] - 28s 35ms/step - loss: 2.0677 - accuracy: 0.3459 - val_loss: 1.4490 - val_accuracy: 0.4999
Epoch 2/40
781/781 [==============================] - 27s 35ms/step - loss: 1.3978 - accuracy: 0.5150 - val_loss: 1.2156 - val_accuracy: 0.5854
Epoch 3/40
781/781 [==============================] - 26s 34ms/step - loss: 1.2044 - accuracy: 0.5746 - val_loss: 1.0826 - val_accuracy: 0.6338
Epoch 4/40
781/781 [==============================] - 27s 34ms/step - loss: 1.0739 - accuracy: 0.6222 - val_loss: 1.0167 - val_accuracy: 0.6615
Epoch 5/40
781/781 [==============================] - 26s 34ms/step - loss: 0.9998 - accuracy: 0.6486 - val_loss: 1.0232 - val_accuracy: 0.6643
Epoch 6/40
781/781 [==============================] - 26s 34ms/step - loss: 0.9489 - accuracy: 0.6657 - val_loss: 0.8523 - val_accuracy: 0.7112
Epoch 7/40
781/781 [==============================] - 27s 34ms/step - loss: 0.9015 - accuracy: 0.6829 - val_loss: 0.8442 - val_accuracy:

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
781/781 [==============================] - 28s 35ms/step - loss: 2.0866 - accuracy: 0.3399 - val_loss: 1.3232 - val_accuracy: 0.5273
Epoch 2/40
781/781 [==============================] - 27s 34ms/step - loss: 1.4383 - accuracy: 0.4956 - val_loss: 1.1999 - val_accuracy: 0.5800
Epoch 3/40
781/781 [==============================] - 27s 34ms/step - loss: 1.2150 - accuracy: 0.5710 - val_loss: 1.2763 - val_accuracy: 0.5849
Epoch 4/40
781/781 [==============================] - 27s 34ms/step - loss: 1.1004 - accuracy: 0.6060 - val_loss: 1.0320 - val_accuracy: 0.6368
Epoch 5/40
781/781 [==============================] - 27s 34ms/step - loss: 1.0055 - accuracy: 0.6427 - val_loss: 1.7466 - val_accuracy: 0.4701
Epoch 6/40
781/781 [==============================] - 27s 34ms/step - loss: 0.9771 - accuracy: 0.6542 - val_loss: 0.9005 - val_accuracy: 0.6902
Epoch 7/40
781/781 [==============================] - 27s 34ms/step - loss: 0.9188 - accuracy: 0.6721 - val_loss: 0.7839 - val_accuracy:

...

Epoch 1/40
781/781 [==============================] - 28s 34ms/step - loss: 2.0968 - accuracy: 0.3366 - val_loss: 1.2008 - val_accuracy: 0.5669
Epoch 2/40
781/781 [==============================] - 26s 34ms/step - loss: 1.4027 - accuracy: 0.5092 - val_loss: 1.1693 - val_accuracy: 0.5998
Epoch 3/40
781/781 [==============================] - 26s 34ms/step - loss: 1.2037 - accuracy: 0.5728 - val_loss: 1.0456 - val_accuracy: 0.6398
Epoch 4/40
781/781 [==============================] - 27s 34ms/step - loss: 1.0862 - accuracy: 0.6157 - val_loss: 0.9204 - val_accuracy: 0.6762
Epoch 5/40
781/781 [==============================] - 27s 34ms/step - loss: 1.0079 - accuracy: 0.6427 - val_loss: 0.8358 - val_accuracy: 0.7075
Epoch 6/40
781/781 [==============================] - 26s 34ms/step - loss: 0.9544 - accuracy: 0.6658 - val_loss: 0.9063 - val_accuracy: 0.6912
Epoch 7/40
781/781 [==============================] - 26s 34ms/step - loss: 0.8979 - accuracy: 0.6842 - val_loss: 0.8782 - val_accuracy:

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
781/781 [==============================] - 28s 35ms/step - loss: 2.0788 - accuracy: 0.3455 - val_loss: 1.3536 - val_accuracy: 0.5377
Epoch 2/40
781/781 [==============================] - 28s 35ms/step - loss: 1.3747 - accuracy: 0.5142 - val_loss: 1.1864 - val_accuracy: 0.5894
Epoch 3/40
781/781 [==============================] - 27s 34ms/step - loss: 1.1936 - accuracy: 0.5798 - val_loss: 1.2512 - val_accuracy: 0.5821
Epoch 4/40
781/781 [==============================] - 27s 34ms/step - loss: 1.0789 - accuracy: 0.6222 - val_loss: 1.0639 - val_accuracy: 0.6559
Epoch 5/40
781/781 [==============================] - 26s 34ms/step - loss: 0.9882 - accuracy: 0.6513 - val_loss: 1.1458 - val_accuracy: 0.6416
Epoch 6/40
781/781 [==============================] - 27s 35ms/step - loss: 0.9527 - accuracy: 0.6682 - val_loss: 0.8259 - val_accuracy: 0.7117
Epoch 7/40
781/781 [==============================] - 27s 35ms/step - loss: 0.8969 - accuracy: 0.6814 - val_loss: 0.8244 - val_accuracy:

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
781/781 [==============================] - 28s 35ms/step - loss: 2.1468 - accuracy: 0.3238 - val_loss: 1.4014 - val_accuracy: 0.5063
Epoch 2/40
781/781 [==============================] - 27s 34ms/step - loss: 1.4526 - accuracy: 0.4890 - val_loss: 1.0913 - val_accuracy: 0.6161
Epoch 3/40
781/781 [==============================] - 27s 34ms/step - loss: 1.2424 - accuracy: 0.5585 - val_loss: 1.1113 - val_accuracy: 0.6187
Epoch 4/40
781/781 [==============================] - 26s 34ms/step - loss: 1.1172 - accuracy: 0.6026 - val_loss: 0.9138 - val_accuracy: 0.6811
Epoch 5/40
781/781 [==============================] - 27s 34ms/step - loss: 1.0217 - accuracy: 0.6382 - val_loss: 0.8778 - val_accuracy: 0.6942
Epoch 6/40
781/781 [==============================] - 26s 34ms/step - loss: 0.9638 - accuracy: 0.6584 - val_loss: 0.8726 - val_accuracy: 0.7001
Epoch 7/40
781/781 [==============================] - 26s 34ms/step - loss: 0.9181 - accuracy: 0.6762 - val_loss: 0.9625 - val_accuracy:

Compare Individual vs Averaged Results

In [ ]:
#ls output/
#grep 'weighted avg' output/*.txt

Evaluate Ensemble

In [9]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from keras.models import load_model
from keras.datasets import cifar10
import numpy as np
import glob
import os

In [12]:
def evaluate_ensemble(models_folder):
    # load the testing data, then scale it into the range [0, 1]
    (testX, testY) = cifar10.load_data()[1]
    testX = testX.astype("float") / 255.0

    # initialize the label names for the CIFAR-10 dataset
    labelNames = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

    # convert the labels from integers to vectors
    lb = LabelBinarizer()
    testY = lb.fit_transform(testY)

    # construct the path used to collect the models then initialize the models list
    modelPaths = os.path.sep.join([models_folder, "*.model"])
    modelPaths = list(glob.glob(modelPaths))
    models = []

    # loop over the model paths, loading the model, and adding it to
    # the list of models
    for (i, modelPath) in enumerate(modelPaths):
        print("[INFO] loading model {}/{}".format(i + 1, len(modelPaths)))
        models.append(load_model(modelPath))

    # initialize the list of predictions
    print("[INFO] evaluating ensemble...")
    predictions = []

    # loop over the models
    for model in models:
        # use the current model to make predictions on the testing data,
        # then store these predictions in the aggregate predictions list
        predictions.append(model.predict(testX, batch_size=64))

    # average the probabilities across all model predictions, then show a classification report
    predictions = np.average(predictions, axis=0)
    print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=labelNames))

In [14]:
evaluate_ensemble(models_folder="drive/MyDrive/pyimagesearch/output/25-ensembles/models")

[INFO] loading model 1/5
[INFO] loading model 2/5
[INFO] loading model 3/5
[INFO] loading model 4/5
[INFO] loading model 5/5
[INFO] evaluating ensemble...
              precision    recall  f1-score   support

    airplane       0.85      0.85      0.85      1000
  automobile       0.90      0.95      0.93      1000
        bird       0.81      0.70      0.75      1000
         cat       0.77      0.52      0.62      1000
        deer       0.76      0.79      0.78      1000
         dog       0.85      0.64      0.73      1000
        frog       0.60      0.96      0.74      1000
       horse       0.88      0.88      0.88      1000
        ship       0.93      0.90      0.92      1000
       truck       0.87      0.92      0.89      1000

    accuracy                           0.81     10000
   macro avg       0.82      0.81      0.81     10000
weighted avg       0.82      0.81      0.81     10000

